In [ ]:
import numpy as np
import gdspy
import matplotlib.pyplot as plt

%matplotlib inline
from functools import partial

In [ ]:
%load_ext autoreload
%autoreload 2
import snakenmake
from snakenmake import *

In [ ]:
def plot_cell(cell, exclude=(2,)):
    # FROM: https://github.com/heitzmann/gdspy/issues/42
    poly_dict = cell.get_polygons(by_spec=True)
    plt.figure(figsize=(20, 12))
    for layer_datatype, polys in poly_dict.items():
        if layer_datatype[0] in exclude:
            continue
        for poly in polys:
            plt.fill(*poly.T, lw=0.5, ec="r", fc="none")
    plt.axes().set_aspect("equal", "datalim")


def write_gds(main_cell, filename, unit=1.0e-6, precision=1.0e-9):
    writer = gdspy.GdsWriter(filename, unit=unit, precision=precision)
    for cell in [main_cell] + list(main_cell.get_dependencies(True)):
        writer.write_cell(cell)
    writer.close()

In [ ]:
coverslip_dims = np.array([55e3, 24e3])
chip_dims = np.array([23e3, 13e3])

In [ ]:
fov = 6.5 * 2048 / 40
fov

In [ ]:
23e3 * 13e3 / fov**2

In [ ]:
import text

a = text.Text("foo", 200)

In [ ]:
b = a[0]

In [ ]:
b.get_polygons()

In [ ]:
import snakenmake
from snakenmake import *
from text import Text as _Text


def Text2(text, size, position=(0, 0), alignment="left", angle=0, **kwargs):
    objs = _Text(text, size, position=(0, 0), **kwargs)
    objs = mirror_refs(objs)
    if angle == 0:
        pass
    elif angle == -np.pi / 2:
        for ref in objs:
            ref.rotation += 90
            ref.origin[:] = ref.origin[::-1]
    elif angle == np.pi / 2:
        for ref in objs:
            ref.rotation -= 90
            ref.origin[:] = ref.origin[::-1] * np.array([1, -1])
    else:
        raise NotImplementedError
    objs = align_refs(objs, position=position, alignment=alignment)
    return objs


main_cell = Cell("main")
# main_cell.add(Text2("foo bar\nbaz taz", 100, position=(5e3,-3e3), angle=np.pi/2, alignment='centered'))
main_cell.add(
    Text2(
        "8Bfoio bar\nbaz taz",
        100,
        position=(5e3, -3e3),
        angle=np.pi / 2,
        alignment="centered",
        layer=6,
    )
)
# main_cell.add(Text2("foo bar\nbaz taz", 100, position=(5e3,-3e3), angle=np.pi/2, alignment='centered', layer=6))
# main_cell.add(Text2("Big snakes\n180503", 1e4, position=(3e4,0), angle=np.pi/2, alignment='centered', layer=6))
plot_cell(main_cell)

In [ ]:
import snakenmake
from snakenmake import *

main_cell = wafer(
    [chip("Big Snake S3 FC90 L35 W1.5 LG20 TS2", snake, split=3, draw_trenches=False)]
    * 6,
    "Big snakes\n180503",
)
# main_cell = chip('Big Snake S3 FC90 L35 W1.5 LG20 TS2', snake, split=3, draw_trenches=False)
plot_cell(main_cell)

In [ ]:
import freetype

In [ ]:
text.DEFAULT_FACE.set_char_size?

In [ ]:
freetype.FT_LOAD_NO_SCALE

In [ ]:
g.CellReference?

# Old

In [ ]:
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            metadata=metadata,
        )
        for fc in (90, 65)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes \n BS/JQS 180510",
)
write_gds(main_cell, "bigsnakes3.gds")

In [ ]:
metadata

In [ ]:
from snakenmake import *

metadata = {}
main_cell = wafer(
    [
        chip(
            "Big Snake S3 FC{fc} L35 W1.5 TS{ts} LG20".format(fc=fc, ts=ts),
            snake,
            split=3,
            feeding_channel_width=fc,
            trench_spacing=ts,
            draw_trenches=True,
            tick_labels=False,
            metadata=metadata,
        )
        for fc in (90,)
        for ts in (1, 1.5, 2)
    ],
    "Big Snakes Backup \n BS/JQS 180508",
)
write_gds(main_cell, "bigsnakes3backup.gds")